In [19]:
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import os, cv2
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import concatenate, Dense, Conv2D, Flatten, Input
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [2]:
path = Path(os.path.abspath(''))

In [4]:
path

WindowsPath('C:/Users/patry/OneDrive/Pulpit/Sieci neuronowe/Music-instrument-recognition/Networks')

In [8]:
#Spectrogram
train_x = np.load(str(path) + '\\train_x.npy')
train_y = np.load(str(path) + '\\train_y.npy')

train_x, test_x, train_y, test_y = train_test_split(train_x, train_y, test_size=0.3, random_state=42)

In [9]:
print(train_x.shape)
print(test_x.shape)

(4693, 128, 130)
(2012, 128, 130)


In [21]:
train_x = train_x.reshape(4693,128,130,1)
test_x = test_x.reshape(2012,128,130,1)

In [22]:
trainInput = Input(shape=(128,130,1))

x = Conv2D(128, kernel_size=3, activation='relu')(trainInput)
x = Conv2D(64, kernel_size=3, activation='relu')(x)
x = Conv2D(32, kernel_size=3, activation='relu')(x)
x = Flatten()(x)
x = Dense(11, activation='sigmoid')(x)

In [23]:
xModel = Model(inputs=trainInput, outputs=x)

In [24]:
#Parameters
data = pd.read_csv(str(path) + '/param_data_train.csv')
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype=float))

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# y_train = to_categorical(y_train)
# y_test = to_categorical(y_test)

In [25]:
print(x_train.shape)
print(x_test.shape)

(4693, 25)
(2012, 25)


In [26]:
trainInput2 = Input(shape=(25,))

y = Dense(128, activation='relu')(trainInput2)
y = Dense(64, activation='relu')(y)
y = Dense(32, activation='relu')(y)
y = Dense(11, activation='sigmoid')(y)

In [27]:
yModel = Model(inputs=trainInput2, outputs=y)

In [28]:
combined = concatenate([xModel.output, yModel.output])

In [29]:
z = Dense(4, activation="relu")(combined)
z = Dense(1, activation="linear")(z)

In [30]:
model = Model(inputs=[xModel.input, yModel.input], outputs=x)
model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])

In [31]:
fit = model.fit([train_x, x_train], train_y, validation_data=([test_x, x_test], test_y), epochs=3)

Train on 4693 samples, validate on 2012 samples
Epoch 1/3
4693/4693 [==============================] - 604s 129ms/step - loss: 0.3738 - accuracy: 0.8930 - val_loss: 0.3046 - val_accuracy: 0.8961
Epoch 2/3
4693/4693 [==============================] - 611s 130ms/step - loss: 0.2453 - accuracy: 0.9097 - val_loss: 0.3208 - val_accuracy: 0.8966
Epoch 3/3
4693/4693 [==============================] - 582s 124ms/step - loss: 0.1492 - accuracy: 0.9438 - val_loss: 0.5242 - val_accuracy: 0.8908


In [32]:
model.reset_metrics()

In [33]:
model.save('model_both.h5')

In [34]:
print(model.input)

[<tf.Tensor 'input_3:0' shape=(None, 128, 130, 1) dtype=float32>, <tf.Tensor 'input_4:0' shape=(None, 25) dtype=float32>]
